<h1 align=center><font size = 5>Predict Grocery Demand for Mobile Grocery Store based on Cuisine Type for Franklin County Neighborhoods in OH</font></h1>


## Introduction


This project is intended to comeup with 5 optimal grocery categories for a mobile grocery store based on Asian Cuisine Types like Chinese, Indian, Thai, Japanese and Korean for Franklin County Neighborhoods in OH. For this project we took Franklin County Neighborhood information from united postal website https://www.unitedstateszipcodes.org/zip-code-database/ with location information and uploaded to the project area.  This project uses Foursquare Venue serach API to get Venus for these neighborhoods for 5 Asian Cuisine Types which is the scope of this project. This project uses K-means approach to cluster the neighborhoods and then we use each cluster mean to calculate the percent of each grocery type need to be packed in the mobile grocery store

**Out of scope:**  This project uses only venue  information to predict the grocery demand not actual demand for the type of grocery .



## Table of Contents
<br/>
<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>  
    <br/>

2. <a href="#item2">Explore Neighborhoods based on Cuisine Category Type in Franklin County OH</a>  
    <br/>

3. <a href="#item3">Analyze Each Neighborhood based on Cuisine Category Type</a>
    <br/>
 
4. <a href="#item4">Cluster Neighborhoods based on Cuisine Category Type</a>  
    <br/>
    
5. <a href="#item5">Examine Clusters</a>    
    <br/>

6. <a href="#item6">Find Inventory Percentage for each Cuisine Category Type</a>  
    <br/>
    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [235]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>
## 1. Download and Explore Dataset

Loading Columbus OH top 10 neighborhood data and creating dataframe

In [236]:
# The code was removed by Watson Studio for sharing.

In [237]:
franklin_neigborhoods = pd.read_csv(body)
franklin_neigborhoods.head()


,Borough,Neighborhood,Latitude,Longitude
0,Franklin County OH,Amlin,40.07,-83.18
1,Franklin County OH,Blacklick,40.02,-82.80
2,Franklin County OH,Dublin,40.10,-83.15
3,Franklin County OH,Dublin,40.11,-83.13
4,Franklin County OH,Hilliard,40.03,-83.14


#### Using geopy library to get the latitude and longitude values of Franklin County OH

In [238]:
address = 'Franklin County OH'

geolocator = Nominatim(user_agent="oh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Franklin OH are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Franklin OH are 40.0283162, -83.0076657.


#### Create a map of Columbus OH  with neighborhoods superimposed on top.

In [239]:
# create map of Franklin County OH using latitude and longitude values
map_franklin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(franklin_neigborhoods['Latitude'], franklin_neigborhoods['Longitude'], franklin_neigborhoods['Borough'], franklin_neigborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_franklin)  
    
map_franklin

#### Foursquare Credentials and Version

In [240]:
# The code was removed by Watson Studio for sharing.

#### Grocery Categories

In [243]:
#Chinese,Thai,Japanese,Korean,Indian Restaurant
categoryIds="4bf58dd8d48988d145941735,4bf58dd8d48988d149941735,4bf58dd8d48988d111941735,4bf58dd8d48988d113941735,4bf58dd8d48988d10f941735"

Exploring one neighborhood 

In [244]:
neighborhood_latitude = franklin_neigborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = franklin_neigborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = franklin_neigborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Amlin are 40.07, -83.18.


In [245]:
# Foursquare URL with categoryIds of Indian, American,Chinese, Mexican and Meditaranian
radius="40300"
limit="1000"
url="https://api.foursquare.com/v2/venues/search?client_id="+str(CLIENT_ID)+"&client_secret="+str(CLIENT_SECRET)+"&ll="+str(neighborhood_latitude)+","+str(neighborhood_longitude)+"&radius="+radius+"&limit="+limit+"&categoryId="+str(categoryIds)+"&v="+VERSION

In [246]:
results = requests.get(url).json()

venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in nearby_venues.columns if col.startswith('location.')] + ['id']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean column names by keeping only last term
nearby_venues.columns = [column.split('.')[-1] for column in nearby_venues.columns]

nearby_venues.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Ampersand Asian Supper Club,Asian Restaurant,940 N High St,US,Columbus,United States,NaN,17925,"[940 N High St, Columbus, OH 43201, United Sta...","[{'label': 'display', 'lat': 39.981694, 'lng':...",39.981694,-83.004151,NaN,43201,OH,5d163f787950f70023787c53
1,Royal Dragon Restaurant,Chinese Restaurant,NaN,US,Columbus,United States,NaN,22387,"[Columbus, OH 43230, United States]","[{'label': 'display', 'lat': 40.08094493077066...",40.080945,-82.917567,NaN,43230,OH,599d9ca9a3061917e57abc4e
2,Erawan Thai,Thai Restaurant,3589 Refugee Rd,US,Columbus,United States,NaN,29327,"[3589 Refugee Rd, Columbus, OH 43232, United S...","[{'label': 'display', 'lat': 39.91640296855650...",39.916403,-82.900614,NaN,43232,OH,4e6012f245ddda35288941de
3,Genji Go,Japanese Restaurant,8599 Columbus Pike,US,Orange,United States,NaN,17350,"[8599 Columbus Pike, Orange, OH, United States]","[{'label': 'display', 'lat': 40.16609149961594...",40.166091,-83.019528,NaN,NaN,OH,5365757a498ecbb366719b64
4,Cumin And Curry Indian Kitchen,Indian Restaurant,4155 Morse Xing,US,Columbus,United States,NaN,22263,"[4155 Morse Xing, Columbus, OH 43219, United S...","[{'label': 'display', 'lat': 40.055145, 'lng':...",40.055145,-82.919400,NaN,43219,OH,5b186fc41acf11002c15d700


In [133]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

50 venues were returned by Foursquare.


<a id='item2'></a>
## 2. Now exploring all 10 neighborhoods in Franklin County OH

In [247]:
def getNearbyVenues(names, latitudes, longitudes, radius=40300): #25 mile radius in meters
    # Foursquare URL with categoryIds of Indian, American,Chinese, Mexican and Meditaranian
    limit="1000"  # limit , but Foursquare gives only 50 venues with developer account 
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            categoryIds,
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [248]:
franklin_neigborhood_venues = getNearbyVenues(names=franklin_neigborhoods['Neighborhood'],
                                   latitudes=franklin_neigborhoods['Latitude'],
                                   longitudes=franklin_neigborhoods['Longitude']
                                  )

In [249]:
print(franklin_neigborhood_venues.shape)
franklin_neigborhood_venues.head()

(6700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amlin,40.07,-83.18,Grand China,40.032928,-83.142803,Chinese Restaurant
1,Amlin,40.07,-83.18,Amul India,40.086612,-83.092769,Indian Restaurant
2,Amlin,40.07,-83.18,Tensuke Express,40.050957,-83.051167,Japanese Restaurant
3,Amlin,40.07,-83.18,Akai Hana,40.050662,-83.051587,Japanese Restaurant
4,Amlin,40.07,-83.18,House of Japan,40.077304,-83.134256,Japanese Restaurant


<a id='item3'></a>
## 3. Analyze Each Neighborhood

In [250]:
# one hot encoding
franklin_onehot = pd.get_dummies(franklin_neigborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
franklin_onehot['Neighborhood'] = franklin_neigborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [franklin_onehot.columns[-1]] + list(franklin_onehot.columns[:-1])
franklin_onehot = franklin_onehot[fixed_columns]

franklin_onehot.head()

,Neighborhood,Asian Restaurant,Chinese Restaurant,Deli / Bodega,Indian Restaurant,Japanese Restaurant,Korean Restaurant,Seafood Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant
0,Amlin,0,1,0,0,0,0,0,0,0,0
1,Amlin,0,0,0,1,0,0,0,0,0,0
2,Amlin,0,0,0,0,1,0,0,0,0,0
3,Amlin,0,0,0,0,1,0,0,0,0,0
4,Amlin,0,0,0,0,1,0,0,0,0,0


In [251]:
franklin_onehot.shape

(6700, 11)

#### Cleaningup additional  categories that are not served by Mobile Restaurents

In [252]:
#American,Asian,Caribbean,Eastern European,Mexican Restaurant
mobileRestaurentsCategories_with_neigborhood=['Neighborhood', 'Chinese Restaurant', 'Thai Restaurant',
       'Japanese Restaurant',
       'Korean Restaurant', 'Indian Restaurant']

In [253]:
dropList=set(franklin_onehot.columns) - set(mobileRestaurentsCategories_with_neigborhood)

In [254]:
franklin_onehot=franklin_onehot.drop(columns=dropList)
franklin_onehot.shape

(6700, 6)

#### Next, grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [255]:
franklin_grouped = franklin_onehot.groupby('Neighborhood').mean().reset_index()
franklin_grouped.head()

,Neighborhood,Chinese Restaurant,Indian Restaurant,Japanese Restaurant,Korean Restaurant,Thai Restaurant
0,Amlin,0.470000,0.100000,0.160000,0.02,0.090000
1,Blacklick,0.460000,0.050000,0.200000,0.02,0.100000
2,Brice,0.480000,0.050000,0.190000,0.02,0.100000
3,Canal Winchester,0.500000,0.060000,0.170000,0.02,0.100000
4,Columbus,0.483478,0.058913,0.164565,0.02,0.098913


In [256]:
franklin_grouped.shape

(15, 6)

In [257]:
num_top_venues = 5

for hood in franklin_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = franklin_grouped[franklin_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amlin----
                 venue  freq
0   Chinese Restaurant  0.47
1  Japanese Restaurant  0.16
2    Indian Restaurant  0.10
3      Thai Restaurant  0.09
4    Korean Restaurant  0.02


----Blacklick----
                 venue  freq
0   Chinese Restaurant  0.46
1  Japanese Restaurant  0.20
2      Thai Restaurant  0.10
3    Indian Restaurant  0.05
4    Korean Restaurant  0.02


----Brice----
                 venue  freq
0   Chinese Restaurant  0.48
1  Japanese Restaurant  0.19
2      Thai Restaurant  0.10
3    Indian Restaurant  0.05
4    Korean Restaurant  0.02


----Canal Winchester----
                 venue  freq
0   Chinese Restaurant  0.50
1  Japanese Restaurant  0.17
2      Thai Restaurant  0.10
3    Indian Restaurant  0.06
4    Korean Restaurant  0.02


----Columbus----
                 venue  freq
0   Chinese Restaurant  0.48
1  Japanese Restaurant  0.16
2      Thai Restaurant  0.10
3    Indian Restaurant  0.06
4    Korean Restaurant  0.02


----Dublin----
                 

In [258]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [259]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = franklin_grouped['Neighborhood']

for ind in np.arange(franklin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(franklin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Amlin,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant
1,Blacklick,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
2,Brice,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
3,Canal Winchester,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
4,Columbus,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
5,Dublin,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant
6,Galloway,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
7,Grove City,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
8,Groveport,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
9,Harrisburg,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant


<a id='item4'></a>
## 4. Cluster Neighborhoods

#### Define cluster size 5 becuase client only wanted 5 categories

In [262]:
# set number of clusters
kclusters = 5

franklin_grouped_clustering = franklin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(franklin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 4, 2, 3, 1, 2, 2, 4], dtype=int32)

In [263]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [264]:

franklin_merged = franklin_neigborhoods

# merge franklin_grouped with toronto_data to add latitude/longitude for each neighborhood
franklin_merged = franklin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

franklin_merged # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Franklin County OH,Amlin,40.07,-83.18,3,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant
1,Franklin County OH,Blacklick,40.02,-82.80,0,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
2,Franklin County OH,Dublin,40.10,-83.15,3,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant
3,Franklin County OH,Dublin,40.11,-83.13,3,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant
4,Franklin County OH,Hilliard,40.03,-83.14,1,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
5,Franklin County OH,New Albany,40.07,-82.80,2,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
6,Franklin County OH,Reynoldsburg,39.95,-82.79,0,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
7,Franklin County OH,Reynoldsburg,39.95,-82.81,0,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
8,Franklin County OH,Westerville,40.12,-82.91,4,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
9,Franklin County OH,Columbus,40.10,-83.01,2,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant


In [265]:
franklin_merged.groupby('Cluster Labels').mean()

,Latitude,Longitude
Cluster Labels,,
0,39.960000,-82.807500
1,39.980000,-83.165000
2,39.975283,-82.969623
3,40.093333,-83.153333
4,39.938000,-82.954000


Visualizing the resulting clusters

In [266]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(franklin_merged['Latitude'], franklin_merged['Longitude'], franklin_merged['Neighborhood'], franklin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>
## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [267]:
franklin_merged.loc[franklin_merged['Cluster Labels'] == 0, franklin_merged.columns[[1] + list(range(5, franklin_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Blacklick,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
6,Reynoldsburg,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
7,Reynoldsburg,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
11,Brice,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant


#### Cluster 2

In [268]:
franklin_merged.loc[franklin_merged['Cluster Labels'] == 1, franklin_merged.columns[[1] + list(range(5, franklin_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Hilliard,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
13,Galloway,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant


#### Cluster 3

In [269]:
franklin_merged.loc[franklin_merged['Cluster Labels'] == 2, franklin_merged.columns[[1] + list(range(5, franklin_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,New Albany,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
9,Columbus,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
14,Grove City,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
15,Groveport,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
18,Groveport,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
19,Groveport,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
20,Groveport,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
21,Groveport,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
22,Columbus,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
23,Columbus,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant


In [270]:
franklin_merged.loc[franklin_merged['Cluster Labels'] == 3, franklin_merged.columns[[1] + list(range(5, franklin_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Amlin,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant
2,Dublin,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant
3,Dublin,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant


In [271]:
franklin_merged.loc[franklin_merged['Cluster Labels'] == 4, franklin_merged.columns[[1] + list(range(5, franklin_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Westerville,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
10,Westerville,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
12,Canal Winchester,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
16,Harrisburg,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant
17,Lockbourne,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant


In [272]:
franklin_final=franklin_merged
franklin_final['Chinese Grocery Percent']=franklin_grouped['Chinese Restaurant']*100
franklin_final['Japanese Grocery Percent']=franklin_grouped['Japanese Restaurant']*100
franklin_final['Indian Grocery Percent']=franklin_grouped['Indian Restaurant']*100
franklin_final['Thai Grocery Percent']=franklin_grouped['Thai Restaurant']*100
franklin_final['Korean Grocery Percent']=franklin_grouped['Korean Restaurant']*100

In [232]:
franklin_final.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Chinese Grocery Percent,Japanese Grocery Percent,Indian Grocery Percent,Thai Grocery Percent,Korean Grocery Percent
0,Franklin County OH,Amlin,40.07,-83.18,3,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant,47.000000,16.000000,10.000000,9.000000,2.0
1,Franklin County OH,Blacklick,40.02,-82.80,0,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant,46.000000,20.000000,5.000000,10.000000,2.0
2,Franklin County OH,Dublin,40.10,-83.15,3,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant,48.000000,19.000000,5.000000,10.000000,2.0
3,Franklin County OH,Dublin,40.11,-83.13,3,Chinese Restaurant,Japanese Restaurant,Indian Restaurant,Thai Restaurant,Korean Restaurant,50.000000,17.000000,6.000000,10.000000,2.0
4,Franklin County OH,Hilliard,40.03,-83.14,1,Chinese Restaurant,Japanese Restaurant,Thai Restaurant,Indian Restaurant,Korean Restaurant,48.347826,16.456522,5.891304,9.891304,2.0


<a id='item6'></a>
## 6. Find Inventory Percentage for each Cuisine Category Type

Below are five groups and percent of inventory for various Cuisine Category that Raj need to pack in his mobile grocery trucks

In [276]:
pd.options.display.float_format = '{:,.2f}'.format
franklin_final.groupby('Cluster Labels').mean()

,Latitude,Longitude,Chinese Grocery Percent,Japanese Grocery Percent,Indian Grocery Percent,Thai Grocery Percent,Korean Grocery Percent
Cluster Labels,,,,,,,
0,39.96,-82.81,48.25,17.50,5.75,10.00,2.00
1,39.98,-83.16,47.67,17.73,5.70,9.95,2.00
2,39.98,-82.97,49.83,16.33,6.83,9.00,2.00
3,40.09,-83.15,48.33,17.33,7.00,9.67,2.00
4,39.94,-82.95,48.33,17.33,6.00,9.67,2.00


Done!